In [1]:
from deps.dobot_api import DobotApiDashboard, DobotApiMove
from deps import utils
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
#Connect to robot sockets. 
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)

In [6]:
utils.report_mode(dash)

Mode: ROBOT_MODE_ENABLE


In [63]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [69]:
dash.DisableRobot()

'0,{},DisableRobot();'

In [10]:
resp = dash.GetPose()
coords = resp.split('{')[1].split('}')[0].split(',')
#print(f'X = {coords[0]}\nY = {coords[1]}\nZ = {coords[2]} ')

In [9]:
resp

'0,{280.900149,-3.714920,84.088142,18.743036,0.000000,0.000000},GetPose();'

In [17]:
coords = [float(coord) for coord in coords[:4]]

In [18]:
coords[:4]

[280.900149, -3.71492, 84.088142, 18.743036]

In [61]:
utils.report_error(dash)

In [53]:
utils.get_pose(dash)

X = 284.598163
Y = -1.856339
Z = 121.098694
r = 51.698734


array([284.598163,  -1.856339, 121.098694,  51.698734])

In [15]:
move.MovL(284.596374, -27.757226, 126.169655,  0)

'0,{},MovL(284.596374,-27.757226,126.169655,0.000000);'

In [20]:
move.JointMovJ(0,0,0,180)

'0,{},JointMovJ(0.000000,0.000000,0.000000,180.000000);'

In [21]:
default_pos = lambda: move.JointMovJ(0,0,0,0)

In [68]:
move.JointMovJ(0,0,0,0)

'0,{},JointMovJ(0.000000,0.000000,0.000000,0.000000);'

In [24]:
dash.DisableRobot()

'0,{},DisableRobot();'

In [64]:
default_pos()

'0,{},JointMovJ(0.000000,0.000000,0.000000,0.000000);'